**Project part 3**

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

import spacy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import tensorflow as tf
import tensorflow_hub as hub
import nltk
import warnings
warnings.simplefilter("ignore")

!wget https://raw.githubusercontent.com/yogawicaksana/helper_prabowo/main/helper_prabowo_ml.py
    
    
from helper_prabowo_ml import clean_html, remove_links, non_ascii, lower, email_address, removeStopWords, punct, remove_, remove_special_characters, remove_digits
import matplotlib.pyplot as plt
import seaborn as sns
import warnings, re
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TFBertModel
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, GlobalMaxPool1D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from sklearn.metrics import classification_report
from tensorflow.keras.utils import plot_model


from helper_prabowo_ml import clean_html, remove_links, non_ascii, lower, email_address, removeStopWords, punct, remove_, remove_special_characters, remove_digits
import matplotlib.pyplot as plt
import seaborn as sns
import warnings, re
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TFBertModel
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, GlobalMaxPool1D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from sklearn.metrics import classification_report
from tensorflow.keras.utils import plot_model

/kaggle/input/emotion/Emotion_final.csv
--2023-12-04 03:39:06--  https://raw.githubusercontent.com/yogawicaksana/helper_prabowo/main/helper_prabowo_ml.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13881 (14K) [text/plain]
Saving to: ‘helper_prabowo_ml.py’

helper_prabowo_ml.p 100%[===================>]  13.56K  --.-KB/s    in 0s      

2023-12-04 03:39:06 (55.9 MB/s) - ‘helper_prabowo_ml.py’ saved [13881/13881]

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


## 1: Exploratory Data Analysis and Preprocessing

In [4]:
import torch
from tqdm.notebook import tqdm

In [5]:
data = pd.read_csv('/kaggle/input/emotion/Emotion_final.csv')
data.head(10)

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
5,ive been feeling a little burdened lately wasn...,sadness
6,ive been taking or milligrams or times recomme...,surprise
7,i feel as confused about life as a teenager or...,fear
8,i have been with petronas for years i feel tha...,happy
9,i feel romantic too,love


Loading the data.

In [6]:
pd.set_option("display.max_colwidth", 240)
data.head()

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplace i will know that it is still on the property,love
4,i am feeling grouchy,anger


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21459 entries, 0 to 21458
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Text     21459 non-null  object
 1   Emotion  21459 non-null  object
dtypes: object(2)
memory usage: 335.4+ KB


Getting information about the data.

In [8]:
data.isnull().sum()

Text       0
Emotion    0
dtype: int64

In [ ]:
Making sure that there is no empty data. 



data['Emotion'].value_counts()

In [10]:
# Show sample
print(f"{data['Text'][0]} -> {data['Emotion'][0]}")

i didnt feel humiliated -> sadness


Getting a sample could be used as example to be trained. 

In [11]:
# load english language model and create nlp object from it
nlp = spacy.load("en_core_web_sm")

In [12]:
txt = data['Text'][3]
txt

'i am ever feeling nostalgic about the fireplace i will know that it is still on the property'

Grabbing a sample from the data. 

In [13]:
doc = nlp(txt)

In [14]:
for token in doc:
    print(token)

i
am
ever
feeling
nostalgic
about
the
fireplace
i
will
know
that
it
is
still
on
the
property


Tokennizing the sample down to word by word.

In [15]:
for token in doc:
    print(f"Word: {token} | -> {token.lemma_}")

Word: i | -> i
Word: am | -> be
Word: ever | -> ever
Word: feeling | -> feel
Word: nostalgic | -> nostalgic
Word: about | -> about
Word: the | -> the
Word: fireplace | -> fireplace
Word: i | -> i
Word: will | -> will
Word: know | -> know
Word: that | -> that
Word: it | -> -PRON-
Word: is | -> be
Word: still | -> still
Word: on | -> on
Word: the | -> the
Word: property | -> property


Using lemminzation to stop meanings to be repeated. 

In [16]:
for token in doc:
    if token.is_stop or token.is_punct:
        print(token)

i
am
ever
about
the
i
will
that
it
is
still
on
the


Taking out the stop words from the sample. 

In [17]:
# use this utility function to get the preprocessed text data
def preprocess(text):
    # remove stop words and lemmatize the text
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    
    return " ".join(filtered_tokens)

In [18]:
print(txt)
procces_txt = preprocess(txt)
print(procces_txt)

i am ever feeling nostalgic about the fireplace i will know that it is still on the property
feel nostalgic fireplace know property


What the sample looks like when it is preprocessed. 

In [19]:
data['preprocessed_text'] = data['Text'].apply(preprocess)

In [20]:
data

,Text,Emotion,preprocessed_text
0,i didnt feel humiliated,sadness,not feel humiliate
1,i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake,sadness,feel hopeless damn hopeful care awake
2,im grabbing a minute to post i feel greedy wrong,anger,be grab minute post feel greedy wrong
3,i am ever feeling nostalgic about the fireplace i will know that it is still on the property,love,feel nostalgic fireplace know property
4,i am feeling grouchy,anger,feel grouchy
...,...,...,...
21454,Melissa stared at her friend in dism,fear,Melissa stare friend dism
21455,Successive state elections have seen the governing parties pummelled by a dismayed electora,fear,successive state election see govern party pummel dismayed electora
21456,Vincent was irritated but not dismay,fear,Vincent irritated dismay
21457,Kendall-Hume turned back to face the dismayed coup,fear,Kendall Hume turn face dismayed coup


Add preprocessed text into the table.Tuned the model with the new preprocessed text.

In [21]:
data['Emotion_num'] = data['Emotion'].map({'happy' : 0, 'sadness': 1, 'anger': 2, 'fear': 3, 'love':4, 'surprise':5})

data.head(10)

,Text,Emotion,preprocessed_text,Emotion_num
0,i didnt feel humiliated,sadness,not feel humiliate,1
1,i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake,sadness,feel hopeless damn hopeful care awake,1
2,im grabbing a minute to post i feel greedy wrong,anger,be grab minute post feel greedy wrong,2
3,i am ever feeling nostalgic about the fireplace i will know that it is still on the property,love,feel nostalgic fireplace know property,4
4,i am feeling grouchy,anger,feel grouchy,2
5,ive been feeling a little burdened lately wasnt sure why that was,sadness,have feel little burden lately not sure,1
6,ive been taking or milligrams or times recommended amount and ive fallen asleep a lot faster but i also feel like so funny,surprise,have take milligram time recommend have fall asleep lot fast feel like funny,5
7,i feel as confused about life as a teenager or as jaded as a year old man,fear,feel confused life teenager jade year old man,3
8,i have been with petronas for years i feel that petronas has performed well and made a huge profit,happy,petronas year feel petronas perform huge profit,0
9,i feel romantic too,love,feel romantic,4


labeling the preprocessed text with numebrs associted with the emotion. 

## 2: Training/Validation Split

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_val, y_train, y_val = train_test_split(data.index.values, 
                                                  data.Emotion_num.values, 
                                                  test_size=0.15, 
                                                  random_state=42,
                                                  stratify=data.Emotion_num.values)

In [24]:
data['data_type'] = ['not_set']*data.shape[0]

In [25]:
data.head()

,Text,Emotion,preprocessed_text,Emotion_num,data_type
0,i didnt feel humiliated,sadness,not feel humiliate,1,not_set
1,i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake,sadness,feel hopeless damn hopeful care awake,1,not_set
2,im grabbing a minute to post i feel greedy wrong,anger,be grab minute post feel greedy wrong,2,not_set
3,i am ever feeling nostalgic about the fireplace i will know that it is still on the property,love,feel nostalgic fireplace know property,4,not_set
4,i am feeling grouchy,anger,feel grouchy,2,not_set


In [26]:
data.loc[X_train, 'data_type'] = 'train'
data.loc[X_val, 'data_type'] = 'val'

Splitting the datat into train and validation. 

In [27]:
data.groupby(['Emotion_num', 'data_type']).count()

Text  Emotion  preprocessed_text
Emotion_num data_type                                  
0           train      5975     5975               5975
            val        1054     1054               1054
1           train      5325     5325               5325
            val         940      940                940
2           train      2544     2544               2544
            val         449      449                449
3           train      2254     2254               2254
            val         398      398                398
4           train      1395     1395               1395
            val         246      246                246
5           train       747      747                747
            val         132      132                132

# 3. Loading Tokenizer and Encoding our Data

In [28]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [29]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

In [30]:
encoded_data_train = tokenizer.batch_encode_plus(
    data[data.data_type=='train'].preprocessed_text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    data[data.data_type=='val'].preprocessed_text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(data[data.data_type=='train'].Emotion_num.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(data[data.data_type=='val'].Emotion_num.values)

In [31]:
dataset_train = TensorDataset(input_ids_train, 
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val, 
                            attention_masks_val,
                           labels_val)

In [32]:
len(dataset_train)

18240

In [33]:
dataset_val.tensors

(tensor([[  101,  2022,  6723,  ...,     0,     0,     0],
         [  101,  2031,  2202,  ...,     0,     0,     0],
         [  101,  2228,  1055,  ...,     0,     0,     0],
         ...,
         [  101, 20006,  2227,  ...,     0,     0,     0],
         [  101, 13014, 14753,  ...,     0,     0,     0],
         [  101, 11165,  2110,  ...,     0,     0,     0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 tensor([2, 5, 2,  ..., 3, 3, 3]))

# 4. Setting up BERT Pretrained Model

In [34]:
from transformers import BertForSequenceClassification

In [35]:
label_dict = data['Emotion_num']

Using bert as the model. Felt this matched up more with the data than the other deep learning models. 

In [36]:
model = BertForSequenceClassification.from_pretrained(
                                      'bert-base-uncased', 
                                      num_labels = len(label_dict),
                                      output_attentions = False,
                                      output_hidden_states = False
                                     )

# 5. Creating Data Loaders

In [37]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [38]:
batch_size = 4

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)

# 6. Setting Up Optimizer and Scheduler

In [39]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [40]:
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
)

In [41]:
epochs = 10 # I tried 15 and 7 and 10 was the better choice

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps = len(dataloader_train)*epochs
)

# 7. Defining our Performance Metrics

In [42]:
import numpy as np
from sklearn.metrics import f1_score

In [43]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')

In [44]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')

# 8. Creating our Training Loop

In [45]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [46]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [47]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [48]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train, 
                        desc='Epoch {:1d}'.format(epoch), 
                        leave=False, 
                        disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        
        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total +=loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})     
    
    #torch.save(model.state_dict(), f'Models/BERT_ft_Epoch{epoch}.model')
    
    tqdm.write('\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')



Epoch {epoch}
Training loss: 1.09532940382962



Validation loss: 0.38349802990324133
F1 Score (weighted): 0.9127796347602727



Epoch {epoch}
Training loss: 0.3004810339841226



Validation loss: 0.3458893785972406
F1 Score (weighted): 0.9139272905284311



Epoch {epoch}
Training loss: 0.2168236325447428


...


Validation loss: 0.5183981682161228
F1 Score (weighted): 0.9258652656296288



Epoch {epoch}
Training loss: 0.03279482886737162



Validation loss: 0.5660736082054517
F1 Score (weighted): 0.9242847844072065



Epoch {epoch}
Training loss: 0.026027534998229386



Validation loss: 0.5820994490418928
F1 Score (weighted): 0.9231803174150481



# 9. Evaluating our Model

In [49]:
accuracy_per_class(predictions, true_vals)

Class: 20267
Accuracy:984/1054

Class: 20735
Accuracy:904/940

Class: 21179
Accuracy:423/449

Class: 21458
Accuracy:348/398

Class: 19988
Accuracy:198/246

Class: 20895
Accuracy:114/132



The accuracy is not to bad with the model. 